<a href="https://colab.research.google.com/github/Engbasemhamada/NYC-Taxi-Fare-Prediction/blob/main/scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Final Internship Task - Data Preprocessing, Model Building, Hyperparameter Tuning
Team: GDSC (Prepared by Eng. Bassem Hamada)
"""

# =============================
# 📦 استيراد المكتبات
# =============================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# تجاهل التحذيرات
import warnings
warnings.filterwarnings("ignore")

# =============================
# 📂 تحميل البيانات
# =============================
data = pd.read_csv("final_internship_data.csv")

print("✅ Data Loaded Successfully!")
print("Shape:", data.shape)
print("Columns:", data.columns.tolist())

# =============================
# 🔍 استكشاف البيانات
# =============================
print("\n📋 عرض أول 5 صفوف:")
print(data.head())

print("\nℹ️ معلومات عن الأعمدة:")
print(data.info())

# =============================
# 🧹 معالجة القيم الفارغة
# =============================
# حساب عدد القيم الفارغة في كل عمود
print("\n🧩 القيم الفارغة قبل المعالجة:")
print(data.isnull().sum())

# حذف الصفوف اللي فيها قيمة مفقودة في العمود المستهدف
data = data[data["fare_amount"].notna()]

# ملء القيم المفقودة الرقمية بالمتوسط
num_cols = data.select_dtypes(include=[np.number]).columns.tolist()
for col in num_cols:
    data[col].fillna(data[col].mean(), inplace=True)

# ملء القيم المفقودة النصية بأكثر قيمة تكرارًا
cat_cols = data.select_dtypes(include=["object"]).columns.tolist()
for col in cat_cols:
    data[col].fillna(data[col].mode()[0], inplace=True)

print("\n✅ بعد المعالجة - لا توجد قيم مفقودة:")
print(data.isnull().sum().sum(), "missing values remaining.")

# =============================
# 🔠 تحويل القيم النصية إلى رقمية
# =============================
le = LabelEncoder()
for col in cat_cols:
    data[col] = le.fit_transform(data[col])

# =============================
# 🎯 تحديد العمود المستهدف
# =============================
target = "fare_amount"
X = data.drop(columns=[target])
y = data[target]

# =============================
# ✂️ تقسيم البيانات إلى تدريب واختبار
# =============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print("\n📊 Train/Test Shapes:", X_train.shape, X_test.shape)

# =============================
# ⚙️ توحيد القيم (Standardization)
# =============================
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print("✅ تم توحيد القيم الرقمية بنجاح")

# =============================
# 🧠 بناء موديل مبدئي (Baseline Model)
# =============================
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

# التنبؤ
y_pred = rf.predict(X_test)

# تقييم الأداء
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("\n📈 Baseline RandomForest Results:")
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")

# =============================
# 🔍 Hyperparameter Tuning
# =============================
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [10, 20, None],
    "min_samples_split": [2, 5, 10]
}

print("\n🚀 بدء عملية ضبط المعاملات (Hyperparameter Tuning)...")

grid = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid,
    cv=3,
    scoring="r2",
    n_jobs=-1
)
grid.fit(X_train, y_train)

print("\n✅ أفضل معاملات تم إيجادها:")
print(grid.best_params_)

# =============================
# 🏁 تقييم النموذج الأفضل
# =============================
best_model = grid.best_estimator_
y_pred_best = best_model.predict(X_test)

best_r2 = r2_score(y_test, y_pred_best)
best_rmse = np.sqrt(mean_squared_error(y_test, y_pred_best))

print("\n🌟 Final Model Performance:")
print(f"Best R² Score: {best_r2:.4f}")
print(f"Best RMSE: {best_rmse:.4f}")

# =============================
# 💾 حفظ النتائج في ملف
# =============================
results = pd.DataFrame({
    "Model": ["RandomForestRegressor"],
    "R2_Score": [best_r2],
    "RMSE": [best_rmse],
    "Best_Params": [str(grid.best_params_)]
})
results.to_csv("model_results_summary.csv", index=False)
print("\n📁 Saved results to 'model_results_summary.csv'")

# =============================
# 🎉 النهاية
# =============================
print("\n✅ Task Completed Successfully by Eng. Bassem Hamada (GDSC Team) ✅")



✅ Data Loaded Successfully!
Shape: (50000, 26)
Columns: ['User ID', 'User Name', 'Driver Name', 'Car Condition', 'Weather', 'Traffic Condition', 'key', 'fare_amount', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'hour', 'day', 'month', 'weekday', 'year', 'jfk_dist', 'ewr_dist', 'lga_dist', 'sol_dist', 'nyc_dist', 'distance', 'bearing']

📋 عرض أول 5 صفوف:
    User ID          User Name       Driver Name Car Condition Weather  \
0  KHVrEVlD     Kimberly Adams        Amy Butler     Very Good   windy   
1  lPxIuEri       Justin Tapia  Hannah Zimmerman     Excellent  cloudy   
2  gsVN8JLS    Elizabeth Lopez    Amanda Jackson           Bad  stormy   
3  9I7kWFgd      Steven Wilson          Amy Horn     Very Good  stormy   
4  8QN5ZaGN  Alexander Andrews  Cassandra Larson           Bad  stormy   

   Traffic Condition      key  fare_amount  pickup_datetime  pickup_longitude  \
0  Congested Traffic  26:21.0          4.5  